## Import libraries
Developing in Jupyter Lab, so using JupyterDash

In [1]:
import pandas as pd
from jupyter_dash import JupyterDash
import dash
from dash import dcc, html
import plotly.express as px
import censoring

## Configure for JupyterDash

In [3]:
JupyterDash.infer_jupyter_proxy_config()

## Filter

In [4]:
df = pd.read_csv(censoring.wordle_output)
# limit to the puzzle in which we last have words (396)
df2 = df[ (df['PuzzleNum'] < 447) & 
         (df['PuzzleNum'] != 421) & 
         (df['Difficulty'] != 'Undefined') &
         (df['Name'] != 'Player7') & 
         (df['Name'] != 'Player8')
        ].copy()

# limit puzzles to where 6 players participated
df3 = df2[['Name','PuzzleNum']].groupby(['PuzzleNum']).agg('count').reset_index()
list_of_puzzles = df3[ df3['Name'] == 6 ]['PuzzleNum'].tolist()
df2 = df2[ df2['PuzzleNum'].isin(list_of_puzzles)].copy()

# make player list
player_list = df2['Name'].unique().tolist()

## Work-in-Progress

In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/dZVMbK.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

app.layout = html.Div(
    children=[
        html.H1(children='Wordle Scoreboard'),
        html.Div(children='Player Score Distribution'),
        html.Div([
            html.Div([
                dcc.Checklist(
                    id='crossfilter-xaxis-column',
                    options=[{'label': i, 'value': i} for i in player_list],
                    inline=True,
                    value=player_list
                ),
                dcc.Graph(id='bars-fail-total')
            ]),
            html.Div([
                dcc.Graph(id='bars-fail-distribution')
            ])
        ],
            style={'width': '49%', 'display': 'flex'}
        )
    ]
)

@app.callback(
    dash.dependencies.Output('bars-fail-distribution', 'figure'),
    dash.dependencies.Input('crossfilter-xaxis-column', 'value')
)
def update_distplot(selected_player):
    # input filters df onto a single name
    filtered_df = df2[ df2.Name.isin(selected_player) ]
    fig = px.histogram(
        filtered_df, 
        x="Fails", 
        # y="Counts", 
        color="Name", 
        # barmode="group"
    )
    # now update the figture with a click
    fig.update_layout(transition_duration=250)

    return fig

@app.callback(
    dash.dependencies.Output('bars-fail-total', 'figure'),
    dash.dependencies.Input('crossfilter-xaxis-column', 'value')
)
def update_barplot(selected_player):
    df3 = df2.groupby(['Name']).agg({'Fails':'sum'}).reset_index()
    filtered_df = df3[ df3.Name.isin(selected_player) ]
    fig = px.bar(
        filtered_df, 
        y='Name', 
        x='Fails', 
        color='Name'
    )
    # update the figure
    fig.update_layout(transition_duration=250)
    
    return fig

if __name__ == '__main__':
    # run better without 'mode="jupyterdash"'
    app.run_server(debug=True, threaded=True)

Dash app running on http://127.0.0.1:8050/
